# Graph

This page will show the knowledge graph.

In [1]:
import os
import re
import json
from pyvis.network import Network

net = Network(height='700px', width='100%', notebook=False, directed=True, cdn_resources='in_line', bgcolor='#0b1020', font_color='#f5f7ff')

node_path = 'nodes'
files = sorted([f for f in os.listdir(node_path) if f.endswith('.ipynb')])

for f in files:
    label = f.replace('.ipynb', '').replace('-', ' ').title()
    url = f"nodes/{f.replace('.ipynb', '.html')}"
    net.add_node(f, label=label, color='#4a90e2', title=f"Click to open {label}", url=url)

    with open(os.path.join(node_path, f), 'r', encoding='utf-8') as file:
        content = file.read()
        links = re.findall(r'\]\((.*?\.ipynb)', content)
        for link in links:
            target = os.path.basename(link)
            if target in files:
                net.add_edge(f, target, color='#888888')

options = {
    "layout": {"improvedLayout": True},
    "nodes": {
        "shape": "dot",
        "size": 18,
        "borderWidth": 2,
        "color": {
            "border": "#ffffff",
            "background": "#4a90e2",
            "highlight": {"border": "#ffffff", "background": "#50e3c2"},
            "hover": {"border": "#ffffff", "background": "#50e3c2"},
        },
        "font": {"size": 16, "color": "#f5f7ff"},
    },
    "edges": {
        "width": 1.5,
        "color": {
            "color": "#888888",
            "highlight": "#50e3c2",
            "hover": "#50e3c2",
        },
        "smooth": {"type": "dynamic"},
    },
    "interaction": {
        "hover": True,
        "tooltipDelay": 150,
        "navigationButtons": True,
    },
    "physics": {
        "stabilization": {"iterations": 250},
        "barnesHut": {
            "gravitationalConstant": -8000,
            "springLength": 140,
            "springConstant": 0.02,
        },
    },
}

net.set_options(json.dumps(options))

net.save_graph("graph_output.html")

with open("graph_output.html", "r") as f:
    html_content = f.read()

click_script = """
    network.on("click", function (params) {
        if (params.nodes.length > 0) {
            var nodeId = params.nodes[0];
            var nodeData = nodes.get(nodeId);
            if (nodeData && nodeData.url) {
                window.parent.location.href = nodeData.url;
            }
        }
    });
"""

html_content = html_content.replace("</body>", f"<script>{click_script}</script></body>")

with open("graph_output.html", "w") as f:
    f.write(html_content)

<iframe src="graph_output.html" width="100%" height="600" style="border:none;"></iframe>